# Finetuning Using Google Gemma's Model

In [1]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━

In [2]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

In [3]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [4]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [6]:
os.environ["WANDB_DISABLED"] = "True"

In [7]:
lora_config = LoraConfig(
    r = 16,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",
)

#Since I wanted to be original, I created my own dataset. It contains several reasons to missout on our friends' bianual meeting and classifying it as resonable or unreasonable. I asked each of my friends to give me 10 reasons and I combined them togather, while trying to avoid duplicates in one dataset that I upploaded to HuggingFace website.

In [8]:
from datasets import load_dataset

data = load_dataset("JohnKh/Reasons-to-miss-friends-meeting")
data = data.map(lambda samples: tokenizer(samples["Excuse for missing friends meeting"]), batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/download/streaming_download_manager.py:784: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


Map:   0%|          | 0/114 [00:00<?, ? examples/s]

In [9]:
data['train']['Excuse for missing friends meeting']

['Ayham is coming',
 'John is coming ',
 'Obai is coming',
 'Saif is coming',
 'Fayez is coming',
 'Ahmad is coming',
 'Majd is coming',
 'Tareq is coming',
 'Qusay is coming',
 'Fouad is coming',
 'Ayham is not coming',
 'John is not coming ',
 'Obai is not coming',
 'Saif is not coming',
 'Fayez is not coming',
 'Ahmad is not coming',
 'Majd is not coming',
 'Tareq is not coming',
 'Qusay is not coming',
 'Fouad is not coming',
 'Not everyone is coming',
 "I don't want to",
 "I don't like it",
 'I am tired',
 'I am bored',
 "I don't have enough money",
 'Money',
 'Sadness',
 'Depression',
 'Lack of water',
 'The weather',
 'It is raining',
 'It is snowing',
 'Weather',
 'Lack of electrisity',
 'Lack of nutrition',
 'Lack of Money',
 'Lack of power',
 'Lack of food',
 'Lack of sleep',
 'Weakness',
 'Lack of love',
 'Emotional drought',
 'Emotional damage',
 'Emotional discontent',
 'Overgrazing',
 'Watching a film',
 'Watching a movie',
 'Watching a Football match',
 'Watching the new

In [10]:
def formatting_func(example):
  listt=[]
  for i in range(114):
    text = f"Excuse: {example['Excuse for missing friends meeting'][i]}; \n Response:{example['Reasonable/Unreasonable'][i]}"
    listt.append(text)
  return listt

In [11]:
data['train']

Dataset({
    features: ['Excuse for missing friends meeting', 'Reasonable/Unreasonable', 'input_ids', 'attention_mask'],
    num_rows: 114
})

In [12]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=60,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        report_to="none"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/114 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [13]:
trainer.train()

Step,Training Loss
1,7.490800
2,7.258900
3,7.133200
4,6.094100
5,6.846300
6,5.761400
7,5.899500
8,5.069400
9,4.311800
10,4.161800


TrainOutput(global_step=60, training_loss=2.433394894997279, metrics={'train_runtime': 71.1515, 'train_samples_per_second': 3.373, 'train_steps_per_second': 0.843, 'total_flos': 36363180490752.0, 'train_loss': 2.433394894997279, 'epoch': 2.11})

In [14]:
text = "Excuse: Bored;"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=10)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Excuse: Bored; 
 Response:Unreasonable; 
 Response


In [15]:
text = "Excuse: Illness;"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=10)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Excuse: Illness; 
 Response:Reasonable; 
 



In [16]:
text = "Excuse: Exam;"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=10)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Excuse: Exam; 
 Response:Reasonable; 
 Response to


In [18]:
text = "Excuse: Money;"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=10)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Excuse: Money; 
 Response:Unreasonable; 
 Response
